In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
import plotly.io
plotly.io.renderers.default = 'notebook_connected'

In [3]:
datestr='5-april-2020'
CASE_URL_FORMAT = 'https://www.health.govt.nz/system/files/documents/pages/covid-19-case-details-update-%s.xlsx'


In [4]:
case_url = CASE_URL_FORMAT % datestr
case_url = 'https://www.health.govt.nz/system/files/documents/pages/covid-casedetialsupdate-5april.xlsx'

In [5]:
today = pd.to_datetime(datestr.replace('-', ' '))

In [6]:
with pd.ExcelFile(case_url) as excel_file:
    cases_dfs = {name: excel_file.parse(i, header=3) for i, name in enumerate(['confirmed', 'probable'])}

In [7]:
def get_case_counts(df, date_name='Report Date', date_col=0):
    return df.groupby([df.columns[date_col], 'DHB']).agg(c=('DHB', 'count')).c.rename_axis([date_name, 'DHB'])

In [8]:
cdf = pd.concat({key: get_case_counts(df) for key, df in cases_dfs.items()}, axis=1, names=['type']).loc[:today - pd.Timedelta(days=1)]

In [9]:
totals_df = cdf.sum(1).rename('cases').reset_index()

In [10]:
px.bar(totals_df, x='Report Date', y='cases', color='DHB')

In [11]:
px.bar(totals_df, color='Report Date', y='cases', x='DHB', color_continuous_scale = px.colors.sequential.algae)

In [16]:
tdf = cdf.unstack('DHB').fillna(0).cumsum().stack('DHB')
tdf['total'] = tdf.sum(1)




In [17]:
ydf = tdf.loc[tdf.index[-1][0], ['confirmed', 'probable']].unstack().reset_index()

In [30]:
tdf[tdf.index.get_loc_level('Canterbury', level='DHB')[0]]

,type,confirmed,probable,total
Report Date,DHB,,,
2020-02-26,Canterbury,0.0,0.0,0.0
2020-02-28,Canterbury,0.0,0.0,0.0
2020-03-02,Canterbury,0.0,0.0,0.0
2020-03-04,Canterbury,0.0,0.0,0.0
2020-03-05,Canterbury,0.0,0.0,0.0
2020-03-06,Canterbury,0.0,0.0,0.0
2020-03-12,Canterbury,0.0,0.0,0.0
2020-03-13,Canterbury,0.0,0.0,0.0
2020-03-14,Canterbury,0.0,0.0,0.0


In [26]:
tdf.loc['Christchurch',:]

DateParseError: Unknown datetime string format, unable to parse: Christchurch

In [18]:
DHBS = ['Northland', 'Waitemata', 'Auckland', 'Counties Manukau', 'Waikato', 'Bay of Plenty', 'Tairawhiti', 'Lakes','Taranaki', "Hawke's Bay", 'Whanganui', 'MidCentral', 'Hutt Valley', 'Capital and Coast', 'Wairarapa', 'Nelson Marlborough', 'West Coast', 'Canterbury',  'South Canterbury', 'Southern']

In [19]:
px.bar(ydf, y='DHB', x=0, orientation='h', color='type', category_orders={'DHB':DHBS})

In [ ]:
xdf = cdf.unstack('DHB').fillna(0).cumsum()
xdf = xdf.stack(['DHB', 'type'])
xdf = xdf[xdf>0].unstack('type')
sdf = xdf.sum(0, level='Report Date').assign(DHB='total')
xdf = xdf.append(sdf.set_index('DHB', append=True), sort=True)
xdf = xdf.assign(total=xdf.sum(1)).reset_index().fillna(0)

In [ ]:
px.line(xdf, x='Report Date', y='total', color='DHB', hover_data=['confirmed', 'probable'], log_y=True)

In [ ]:
fig=px.bar(xdf[xdf.DHB!='total'], x='Report Date', y='total', color='DHB')
fig.data[0]

In [ ]:
df.groupby('DHB').count()

In [ ]:
cdf.groupby(['Report Date', 'DHB']).agg(count=('DHB', 'count')).cumsum()

In [ ]:
px.bar(cdf.groupby(['Report Date', 'DHB']).agg(count=('DHB', 'count')).reset_index(), x='Report Date', y='count', color='DHB')

In [ ]:
df2.groupby('Report Date').agg(count=('DHB', 'count'))

In [ ]:
df2.groupby('DHB').count()